In [2]:
import pandas as pd
import dash
from dash import html
from dash import dcc
from dash.dependencies import Input, Output
import plotly.express as px

In [4]:
# Read the airline data into pandas dataframe

url = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv"

spacex_df = pd.read_csv(url)
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

In [8]:
spacex_df.head(3)

,Unnamed: 0,Flight Number,Launch Site,class,Payload Mass (kg),Booster Version,Booster Version Category
0,0,1,CCAFS LC-40,0,0.0,F9 v1.0 B0003,v1.0
1,1,2,CCAFS LC-40,0,0.0,F9 v1.0 B0004,v1.0
2,2,3,CCAFS LC-40,0,525.0,F9 v1.0 B0005,v1.0


In [5]:
launch_site_list = spacex_df["Launch Site"].unique().tolist()

In [6]:
launch_site_class = spacex_df.groupby("Launch Site")["class"].mean().reset_index()
total_launch_site_class = spacex_df.groupby("Launch Site")["class"].sum().reset_index()

launch_site_class

,Launch Site,class
0,CCAFS LC-40,0.269231
1,CCAFS SLC-40,0.428571
2,KSC LC-39A,0.769231
3,VAFB SLC-4E,0.400000


In [7]:
# Create a dash application
app = dash.Dash(__name__)

# Create an app layout
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),
                                
                                dcc.Dropdown(
                                    id='site-dropdown',
                                            options=[{'label': 'All Sites', 'value': 'All'},
                                                     {'label': launch_site_list[0], 'value': launch_site_list[0]},
                                                     {'label': launch_site_list[1], 'value': launch_site_list[1]},
                                                     {'label': launch_site_list[2], 'value': launch_site_list[2]},
                                                     {'label': launch_site_list[3], 'value': launch_site_list[3]}
                                                    ],
                                value = 'All',
                                placeholder = 'Select Launch Site',
                                searchable = True),
                                html.Br(),                            


                                html.Div(dcc.Graph(id='success-pie-chart')),
                                html.Br(),

                                html.P("Payload range (Kg):"),
                                dcc.RangeSlider(
                                    id='payload-slider',
                                    min = 0,
                                    max = 10000,
                                    step = 1000,
                                    marks = {i: f'{i}' for i in range(0, 11000, 2500)},
                                    value = [spacex_df["Payload Mass (kg)"].min(),spacex_df["Payload Mass (kg)"].max()]
                                ),

                                # TASK 4: Add a scatter chart to show the correlation between payload and launch success
                                html.Div(dcc.Graph(id='success-payload-scatter-chart')),
                                ])

# TASK 4:
# Add a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output



@app.callback(Output(component_id = 'success-pie-chart', component_property = 'figure'),
              Input(component_id = 'site-dropdown', component_property = 'value'))

def get_pie_chart(entered_site):
    
    if entered_site == 'All':
        fig = px.pie(
            total_launch_site_class, 
            values = 'class',
            names = 'Launch Site',
            title = 'Total Successful Launches by Site')
        
    else:

        site_df = spacex_df[spacex_df["Launch Site"] == entered_site]
        success_count = site_df["class"].sum() 
        failure_count = len(site_df) - success_count
        
        site_row = launch_site_class[launch_site_class["Launch Site"] == entered_site].iloc[0]
        success_rate = site_row['class']
        failure_rate = 1 - success_rate

        outcome_df = pd.DataFrame({
            'Outcome': ['Success', 'Failure'],
            'Rate': [success_rate, failure_rate],
            'Count': [success_count, failure_count]
        })
        
        fig = px.pie(
            outcome_df, 
            values = 'Count',
            names = 'Outcome',
            title = f'Success vs Failure for {entered_site}',
            color = 'Outcome',
            color_discrete_map = {'Success': 'blue', 'Failure': 'orange'}
        )
        
    fig.update_traces(textinfo='percent+value')
    return fig


@app.callback(Output(component_id = 'success-payload-scatter-chart', component_property = 'figure'),
              Input(component_id = 'site-dropdown', component_property = 'value'),
              Input(component_id = 'payload-slider', component_property = 'value'))

def plot_scatter(entered_site, payload_range):

    low, high = payload_range

    filtered_df = spacex_df[
        (spacex_df['Payload Mass (kg)'] >= low) &
        (spacex_df['Payload Mass (kg)'] <= high)
    ]

    if entered_site == 'All':
        fig = px.scatter(
                filtered_df,
                x = 'Payload Mass (kg)',
                y = 'class',
                color = 'Launch Site',
                title = "Payload Mass vs Launch Outcome for All Sites"
        )


    else:

        site_df = spacex_df[spacex_df['Launch Site'] == entered_site].copy()
        filtered_df = site_df[
            (site_df['Payload Mass (kg)'] >= low) &
            (site_df['Payload Mass (kg)'] <= high)
        ].copy()

        
        filtered_df["Outcome"] = filtered_df["class"].map({1: "Success", 0: "Failure"})

        fig = px.scatter(
                filtered_df,
                x = 'Payload Mass (kg)',
                y = 'class',
                color = 'Booster Version',
                #color_discrete_map = {'Success': 'blue', 'Failure': 'red'},
                title = f"Launch Success at {entered_site} by Payload Mass"
        )
        

    fig.update_layout(
        yaxis=dict(range=[-0.1, 1.1])
    )
    return fig
        
                         


# Run the app
if __name__ == '__main__':
    app.run()
